<a href="https://colab.research.google.com/github/Ashmaan-2006/facial-detection/blob/main/facial_keypoint_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('/content/training.csv')  # Update path if needed

print("Dataset shape:", df.shape)
df.head()

# Drop incomplete rows
df = df.dropna()

# Process images
X = np.vstack(df['Image'].apply(lambda im: np.fromstring(im, sep=' ')).values)
X = X.reshape(-1, 96, 96, 1).astype('float32') / 255.0  # Normalize pixels

# Process keypoints
y = df[df.columns[:-1]].values
y = (y - 48) / 48  # Normalize to [-1, 1]

print(f"Images shape: {X.shape}")
print(f"Keypoints shape: {y.shape}")
